In [ ]:
import pandas as pd
import numpy as np

In [ ]:
swap_rates = pd.read_csv("../data/market-data-swap-rates.csv")
# swap_rates.head(50)

In [ ]:
def ema(gp):
    #  Date is in the order natrually, so no need to rearrange the date
    N = len(gp)
    alpha = 2 / (N + 1)
    coe = []
    for i in range(N):
        coe.append(alpha * ((1 - alpha) ** i))
    coe.reverse()
    # print(coe)
    return sum(np.array(gp) * np.array(coe))


In [ ]:
gp_srs = swap_rates.groupby(['Start Date', 'Tenor'])['Swap Rate'].apply(ema).reset_index()
gp_srs

,Start Date,Tenor,Swap Rate
0,2022-01-13,10y,1.081333
1,2022-01-13,15y,1.265960
2,2022-01-13,18M,0.221625
3,2022-01-13,1y,0.195474
4,2022-01-13,2y,0.279370
...,...,...,...
28480,2039-01-12,2y,3.424505
28481,2039-01-12,3y,3.393758
28482,2039-01-12,4y,3.358897
28483,2039-01-12,5y,3.319531


In [ ]:
swap_vols = pd.read_csv("../data/market-data-swaption-vols.csv")
swap_vols["Date"].value_counts()

2022-04-08    810
2022-06-07    810
2023-10-30    810
2022-03-09    810
2021-12-09    810
             ... 
2023-09-28    405
2023-02-14    405
2021-11-19    405
2022-07-21    405
2023-02-13    405
Name: Date, Length: 754, dtype: int64

In [ ]:
# 按照strike和expiration分组，并计算平均波动率
gp_vols = swap_vols.groupby(['Expiry', 'Tenor', 'Date'])['Vols'].mean().reset_index()
gp_vols

,Expiry,Tenor,Date,Vols
0,10y,10y,2021-01-13,0.145596
1,10y,10y,2021-01-15,0.145449
2,10y,10y,2021-01-18,0.145109
3,10y,10y,2021-01-19,0.144115
4,10y,10y,2021-01-20,0.143333
...,...,...,...,...
61069,8y,8y,2024-01-08,0.256855
61070,8y,8y,2024-01-09,0.256540
61071,8y,8y,2024-01-10,0.256868
61072,8y,8y,2024-01-11,0.258929


In [ ]:
trade_information = pd.read_csv("../data/trade-information.csv")
# trade_information.head(20)

In [ ]:
vegas = pd.read_csv("../data/trade-price-ir-vegas.csv")

# Data Combination of vegas and swap rates sheet and the volities sheet

In [ ]:
vegas.head(10)

,Value Date,Trade Name,Trade Currency,Zero Rate Shock,TV,Expiry Bucket,Expiry Date,Tenor Bucket,Vega
0,2022-09-02,dummyTrade1,USD,-100,-227907.098775,1y,2023-09-04,10y,1.962246
1,2022-09-02,dummyTrade1,USD,-50,-222208.400967,1y,2023-09-04,10y,-3.812341
2,2022-09-02,dummyTrade1,USD,-25,-218960.927995,1y,2023-09-04,10y,4.471006
3,2022-09-02,dummyTrade1,USD,-10,-216872.430106,1y,2023-09-04,10y,4.333398
4,2022-09-02,dummyTrade1,USD,-5,-216146.310328,1y,2023-09-04,10y,5.679687
5,2022-09-02,dummyTrade1,USD,0,-215402.656152,1y,2023-09-04,10y,5.016274
6,2022-09-02,dummyTrade1,USD,5,-214640.494071,1y,2023-09-04,10y,3.458172
7,2022-09-02,dummyTrade1,USD,10,-213858.452424,1y,2023-09-04,10y,3.132444
8,2022-09-02,dummyTrade1,USD,25,-211345.804723,1y,2023-09-04,10y,12.184058
9,2022-09-02,dummyTrade1,USD,50,-206384.346001,1y,2023-09-04,10y,15.648409


In [ ]:
V_vegas = pd.merge(vegas, gp_vols, left_on = ["Value Date", "Expiry Bucket", "Tenor Bucket"], \
                right_on= ["Date", "Expiry", "Tenor"], how = "left")

V_vegas = V_vegas.drop(columns = ['Expiry', "Tenor", "Date"])
V_vegas

,Value Date,Trade Name,Trade Currency,Zero Rate Shock,TV,Expiry Bucket,Expiry Date,Tenor Bucket,Vega,Vols
0,2022-09-02,dummyTrade1,USD,-100,-227907.098775,1y,2023-09-04,10y,1.962246,0.223827
1,2022-09-02,dummyTrade1,USD,-50,-222208.400967,1y,2023-09-04,10y,-3.812341,0.223827
2,2022-09-02,dummyTrade1,USD,-25,-218960.927995,1y,2023-09-04,10y,4.471006,0.223827
3,2022-09-02,dummyTrade1,USD,-10,-216872.430106,1y,2023-09-04,10y,4.333398,0.223827
4,2022-09-02,dummyTrade1,USD,-5,-216146.310328,1y,2023-09-04,10y,5.679687,0.223827
...,...,...,...,...,...,...,...,...,...,...
3919099,2024-01-12,dummyTrade9,USD,10,-547852.055930,15y,2039-01-12,8y,5.007952,0.267817
3919100,2024-01-12,dummyTrade9,USD,25,-521228.587618,15y,2039-01-12,8y,5.131505,0.267817
3919101,2024-01-12,dummyTrade9,USD,50,-464905.982097,15y,2039-01-12,8y,3.348622,0.267817
3919102,2024-01-12,dummyTrade9,USD,75,-391315.041929,15y,2039-01-12,8y,1.063634,0.267817


In [ ]:
V_vegas2 = pd.merge(V_vegas, gp_srs, left_on = ["Value Date", "Tenor Bucket"], \
                right_on= ["Start Date", "Tenor"], how = "left")
V_vegas2.drop(columns= ["Tenor", "Start Date"])
V_vegas2

,Value Date,Trade Name,Trade Currency,Zero Rate Shock,TV,Expiry Bucket,Expiry Date,Tenor Bucket,Vega,Vols,Start Date,Tenor,Swap Rate
0,2022-09-02,dummyTrade1,USD,-100,-227907.098775,1y,2023-09-04,10y,1.962246,0.223827,2022-09-02,10y,1.554418
1,2022-09-02,dummyTrade1,USD,-50,-222208.400967,1y,2023-09-04,10y,-3.812341,0.223827,2022-09-02,10y,1.554418
2,2022-09-02,dummyTrade1,USD,-25,-218960.927995,1y,2023-09-04,10y,4.471006,0.223827,2022-09-02,10y,1.554418
3,2022-09-02,dummyTrade1,USD,-10,-216872.430106,1y,2023-09-04,10y,4.333398,0.223827,2022-09-02,10y,1.554418
4,2022-09-02,dummyTrade1,USD,-5,-216146.310328,1y,2023-09-04,10y,5.679687,0.223827,2022-09-02,10y,1.554418
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3919099,2024-01-12,dummyTrade9,USD,10,-547852.055930,15y,2039-01-12,8y,5.007952,0.267817,2024-01-12,8y,2.044383
3919100,2024-01-12,dummyTrade9,USD,25,-521228.587618,15y,2039-01-12,8y,5.131505,0.267817,2024-01-12,8y,2.044383
3919101,2024-01-12,dummyTrade9,USD,50,-464905.982097,15y,2039-01-12,8y,3.348622,0.267817,2024-01-12,8y,2.044383
3919102,2024-01-12,dummyTrade9,USD,75,-391315.041929,15y,2039-01-12,8y,1.063634,0.267817,2024-01-12,8y,2.044383


In [ ]:
V_vegas2.to_csv("Vegas_Vol_SwapRates.csv")

# Data Combination of vegas and dummy trade information

In [ ]:
import pandas as pd

# 完整的csv文件路径
dummyTrade_file_path = '../data/trade-information.csv'
vegas_file_path = '../data/trade-price-ir-vegas.csv'

# 使用panadas的read_csv()函数读取cvs文件，并将其存储为dataframe
dummyTrade_info = pd.read_csv(dummyTrade_file_path)
print(dummyTrade_info.head(5))
vegas_data = pd.read_csv(vegas_file_path)
print(vegas_data.head(5))

In [ ]:
# 将pay_frequency字符串映射为相应的数值
pay_frequency_mapping = {
    '6M': 0.5,
    '4M': 0.333,
    '3M': 0.25
}
dummyTrade_info['pay_frequency'] = dummyTrade_info['pay_frequency'].map(pay_frequency_mapping)

# 将maturity字符串映射为相应的数值
maturity_mapping = {
    '5Y': 5,
    '10Y': 10
}
dummyTrade_info['maturity'] = dummyTrade_info['maturity'].map(maturity_mapping)

print("Transformed dummyTrade_info results:")
print(dummyTrade_info.head(8))

In [ ]:
# 使用 merge() 函数合并两个表格，根据 Trade Name 进行连接
merged_data = pd.merge(vegas_data, dummyTrade_info, how='left', left_on='Trade Name', right_on='trade name')

# 删除 'trade name' 列
merged_data.drop(columns=['trade name'], inplace=True)

print("Merged datasets results:")
print(merged_data.head(3))

# 将数据保存到新的csv文件中，文件名格式为merged_data.csv
filename = f"merged_data.csv"
merged_data.to_csv(filename, index=False)
print(f"Saved {filename}")